In [1]:
:load ./learning-2022/files/LectureNotes/Sections/interpreter/Parsing

### Syntax

In [2]:
data Register = ACC
              | NIL
           -- | ANY
           -- | LAST
              | IN
              | OUT
           -- non-addressable:
              | BAK
              | IP
              deriving (Show, Read, Eq)

data Operand = Reg Register
             | Con Int
             deriving (Show, Read)

type Location = Int           

data Operation = NOP
               | MOV Operand Register
               | SWP
               | SAV
               | ADD Operand
               | SUB Operand
               | NEG
               | JMP Location
               | JEZ Location
               | JNZ Location
               | JGZ Location
               | JLZ Location
            -- | JRO Operand
               deriving (Show, Read)

type Node = [Operation]

### Parsing

In [3]:
import Text.Read
import Data.Maybe
import Control.Monad

import Parsing

In [4]:
word = token $ many upper

register :: Parser Register
register = do xs <- word
              guard $ isJust (readMaybe xs :: Maybe Register)
              return (read xs :: Register)

operand :: Parser Operand
operand =  Reg <$> register
       <|> Con <$> integer

operation :: Parser Operation
operation =  do xs <- word
                guard $ isJust (readMaybe xs :: Maybe Operation)
                return (read xs :: Operation)
         <|> do symbol "MOV"
                o <- operand
                MOV o <$> register
         <|> do w <- word
                guard $ elem w ["ADD","SUB"]
                o <- operand
                return (read (w ++ " (" ++ show o ++ ")") :: Operation) -- messy
         <|> do w <- word
                guard $ elem w ["JMP","JEZ","JNZ","JGZ","JLZ"]
                i <- integer
                return (read (w ++ " (" ++ show i ++ ")") :: Operation)
                
node = many operation

parseNode :: String -> Node
parseNode xs = case parse node xs of
                   [(p , [])] -> p
                   [(_ , s)]  -> error ("syntax: unparsed string " ++ s)
                   _          -> error "syntax: failed to parse"


### Interpreting

In [5]:
type State = Register -> Int

init :: State
init _ = 0

update :: Register -> Int -> State -> State
update r x s = s'
 where s' r' | r' == r   = x
             | otherwise = s r'

eval :: Operand -> State -> Int
eval (Reg r) s = s r
eval (Con x) _ = x

exec :: Operation -> State -> State
exec  NOP      s = s
exec (MOV o r) s = update r (eval o s) s
exec  SWP      s = update BAK (s ACC) $ update ACC (s BAK) s
exec  SAV      s = update BAK (s ACC) s
exec (ADD o)   s = update ACC (s ACC + eval o s) s
exec (SUB o)   s = update ACC (s ACC - eval o s) s
exec  NEG      s = update ACC (negate (s ACC)) s
exec (JMP l)   s = update IP l s
exec (JEZ l)   s = if s ACC == 0 then update IP l s else s
exec (JNZ l)   s = if s ACC /= 0 then update IP l s else s
exec (JGZ l)   s = if s ACC >  0 then update IP l s else s
exec (JLZ l)   s = if s ACC <  0 then update IP l s else s

run :: Node -> Int -> Int
run n i = run' n (update IN i init) OUT 

run' :: Node -> State -> State
run' n s = if s IP >= length n then s else
           run' n $ exec (n !! s IP) $ update IP (s IP + 1) s

### Testing

In [6]:
exampleNode = unlines [
              "MOV IN  ACC",
              "SWP        ",
              "MOV 1   ACC",
              "           ",
              "SWP        ", -- line 3
              "JEZ 9      ",
              "SUB 1      ",
              "SWP        ",
              "ADD ACC    ",
              "JMP 3      ",
              "           ",
              "SWP        ", -- line 9
              "MOV ACC OUT"]

putStr exampleNode

MOV IN  ACC
SWP        
MOV 1   ACC
           
SWP        
JEZ 9      
SUB 1      
SWP        
ADD ACC    
JMP 3      
           
SWP        
MOV ACC OUT

In [7]:
parsedNode = parseNode exampleNode
parsedNode

[MOV (Reg IN) ACC,SWP,MOV (Con 1) ACC,SWP,JEZ 9,SUB (Con 1),SWP,ADD (Reg ACC),JMP 3,SWP,MOV (Reg ACC) OUT]

In [8]:
map (run parsedNode) [0..9]

[1,2,4,8,16,32,64,128,256,512]